In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
pd.options.display.max_rows = 13

In [8]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [9]:
dados = pd.read_excel('Iphone_CDados.xlsx')

In [10]:
dados

,Treinamento,Filtro
0,"@kayllanefer bateria de iphone é uma merda, ja...",1
1,cabo de iphone eh feito pra estragar pqp,1
2,@johnpvx melhor iphone de sempre. não me arrep...,1
3,alguém querendo comprar um cabo/carregador de ...,1
4,to sem carregador já tava sofrendo aqui daí le...,1
...,...,...
328,acho que amanhã lança o iphone 12 👀👀👀,0
329,@pirucaking @paulocoelho não dá pra levar a sé...,0
330,"to querendo comprar o iphone 11pro max, para m...",0
331,triste tirar foto só com iphone 😔 pois é a úni...,0


In [16]:
Relevante = dados["Filtro"].isin(["1"])
Irrelevante = dados["Filtro"].isin(["0"])
Relevante = dados.loc[Relevante, :]
Irrelevante = dados.loc[Irrelevante, :]

In [36]:
Treino_Relevante = Relevante["Treinamento"]
Lista_Relevante = Treino_Relevante.to_string()
print(Lista_Relevante)


0      @kayllanefer bateria de iphone é uma merda, ja...
1               cabo de iphone eh feito pra estragar pqp
2      @johnpvx melhor iphone de sempre. não me arrep...
3      alguém querendo comprar um cabo/carregador de ...
4      to sem carregador já tava sofrendo aqui daí le...
5      @sparktfly iphone 11 miga, não desvaloriza nunca.
6                      @dhyuugaa no iphone buga q ódio 🤡
7                       bateria do iphone, pq tão merda?
8      o iphone já é caro com um símbolo de uma maçã,...
9      rt @isaacauggusto: ah pronto, agora quero o ip...
10     @clopes24 todo iphone fica merda depois de 2 a...
11     não consigo enviar vídeo longo por wpp pra min...
12     tá ligado aquele medo de atualizar o iphone e ...
13     quase quebrei um iphone x e por um segundo já ...
14     @tyrant2kk mas assim, o a10 é melhor e ainda p...
15     tanta gente me seguindo conseguiram fazer o ip...
16                      odeio o leitor de tela do iphone
17     esqueci q iphone trava q